In [1]:
import os
import random
import time
import requests
import json
import csv
import time
import random
import openai
from dotenv import load_dotenv
from notion_client import Client
from pprint import pprint
import httpx
from urllib3.exceptions import InsecureRequestWarning
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


# 设置代理
proxies = {
    "http://": "http://localhost:2112",
    "https://": "http://localhost:2112",
}
# 创建自定义的HTTPX客户端
httpx_client = httpx.Client(proxies=proxies)
# 传入notion client对象
notion = Client(auth=os.environ["NOTION_TOKEN"], client=httpx_client)


# 从.env文件中导入账号(ACCOUNT)和密码(PASSWORD)
dotenv_path = "../../.env"
load_dotenv(dotenv_path)
NOTION_PAGEID = os.getenv("NOTION_PAGEID")

# 设置OpenAI的API Key
openai.api_key = os.getenv("V3_APIKEY")
openai.base_url = "https://api.gpt.ge/v1/"
openai.default_headers = {"x-foo": "true"}

# 忽略不安全请求的警告
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

print("init & import done")

init & import done


获取Notion中以Page格式存储的页面

In [2]:
page_block_list = notion.blocks.children.list(**{
    "block_id": "2d6dd347e9de4c57a5c00aa2ead7299e" # page_id can be used to search children blocks.
})

response = page_block_list["results"]
pprint(response)

[{'archived': False,
  'child_page': {'title': '2024-09-09 '},
  'created_by': {'id': '180a057d-bd2a-43dc-8cfb-ee9f7e93ef08',
                 'object': 'user'},
  'created_time': '2024-09-08T08:40:00.000Z',
  'has_children': True,
  'id': 'f17906ec-24c4-4a72-a36c-596b3cad65ef',
  'in_trash': False,
  'last_edited_by': {'id': '180a057d-bd2a-43dc-8cfb-ee9f7e93ef08',
                     'object': 'user'},
  'last_edited_time': '2024-09-09T08:54:00.000Z',
  'object': 'block',
  'parent': {'page_id': '2d6dd347-e9de-4c57-a5c0-0aa2ead7299e',
             'type': 'page_id'},
  'type': 'child_page'},
 {'archived': False,
  'child_page': {'title': '2024-09-10 '},
  'created_by': {'id': '180a057d-bd2a-43dc-8cfb-ee9f7e93ef08',
                 'object': 'user'},
  'created_time': '2024-09-10T03:02:00.000Z',
  'has_children': True,
  'id': '0de45068-9241-43a1-abf6-a85547379a5c',
  'in_trash': False,
  'last_edited_by': {'id': '180a057d-bd2a-43dc-8cfb-ee9f7e93ef08',
                     'object': 

循环读取除了最后一页下的所有子块，保存到变量中

In [3]:
def search_child_block(id): 
    """
    获取指定id的子块
    :param id: 块或者页面ID
    :return: 
    """
    resp = notion.blocks.children.list(**{
        "block_id": id # page_id can be used to search children blocks.
    })
    return resp["results"]

def get_page_properties(id, property_name): 
    """
    获取指定id页面的对应属性
    :param id: 块或者页面ID
    :return: 
    """
    resp = notion.pages.properties.retrieve(**{
        "page_id": id,
        "property_id": "title"
    })
    return resp["results"][0][property_name]

def get_all_text_blocks(block_id, indent_level=0):
    """
    递归查找所有文本块，并拼接到md_context中
    :param block_id: 块或者页面ID
    :param indent_level: 缩进级别
    :return: 拼接后的文本内容
    """
    blocks = search_child_block(block_id)
    text_content = ""
    for block in blocks:
        if "paragraph" in block and block["paragraph"]["rich_text"]:
            indent = " " * (indent_level * 4)
            text_content += indent + block["paragraph"]["rich_text"][0]["text"]["content"] + "\n"
        if block["has_children"]:
            text_content += get_all_text_blocks(block["id"], indent_level + 1)
    return text_content

md_context = ""

print("func init done")

func init done


获取所有页面下的文本内容

In [4]:
for page in response[:-1]:
    title = get_page_properties(page['id'], "title")
    md_context += "### " + title["plain_text"] + "\n"
    md_context += get_all_text_blocks(page['id'])
print("md_context gen done")

md_context gen done


调用claude的claude 3.5 sonnet模型，生成周报

In [10]:
template = """
"""

rule = """
在编写周报时，请注意以下几点：
1. 请不要以“我们”为第一叙述人，而是以“我”为第一叙述人，
2. 请不要简单的将每天所做过的事情简单罗列出来，而是将重点放在总结与记叙中
3. 请不要在最后给全篇文章总结，干净利落的结束就好，就像我给出的周报模板一样

以下是这周的工作内容：
"""


prompt = "今天是xxxx年xx月xx日，正式实习的第x周，请模仿下面我自己编写的周报文笔风格，根据我这周的工作内容，编写本周周报：" + template + rule + md_context
# pprint(md_context)

# !IMPORTANT 请关闭系统代理再执行该部分代码
completion = openai.chat.completions.create(
    model="claude-3-5-sonnet-20240620",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
)

print("gen result text done!")

KeyboardInterrupt: 

In [9]:

# 将生成的周报保存到./result.md中
with open("./result.md", "w", encoding="utf-8") as f:
    f.write(completion.choices[0].message.content)
print("write to file done!")

write to file done!


TODO 上传周报到Notion